In [40]:
import requests
import pandas as pd
import json

In [2]:
api_key = open("caselaw-key", "r").read().strip()

In [32]:
# Import a csv of headnotes as a pd dataframe
# Drop last row, which is just a copyright footer from Reuters
# wl-headnotes-sample.csv: 20 headnotes of type 143(2) - Existence of Ambiguity (Federal courts, prior to 2016)
headnotes = pd.read_csv("wl-headnotes-sample.csv")[:-1]

In [33]:
# We'll use case citations pulled from WL to make our case.law search queries
# (Citations, unlike docket #s are unique: see https://libguides.law.ucla.edu/citations)
headnotes['Citation']

0      154 F.Supp.3d 189
1      154 F.Supp.3d 189
2      154 F.Supp.3d 189
3      154 F.Supp.3d 453
4      162 F.Supp.3d 200
5      162 F.Supp.3d 200
6     151 F.Supp.3d 1043
7     151 F.Supp.3d 1294
8     151 F.Supp.3d 1294
9      150 F.Supp.3d 228
10          807 F.3d 952
11          550 B.R. 676
12          550 B.R. 676
13         806 F.3d 1217
14          541 B.R. 259
15          541 B.R. 259
16     146 F.Supp.3d 972
17     146 F.Supp.3d 568
18     146 F.Supp.3d 568
19          541 B.R. 467
Name: Citation, dtype: object

In [34]:
# Note that a single case can be associated with multiple headnotes (incl. multiple headnotes of a single type)
# So, make an array of unique citations 
citations = headnotes['Citation'].unique()
citations

array(['154 F.Supp.3d 189', '154 F.Supp.3d 453', '162 F.Supp.3d 200',
       '151 F.Supp.3d 1043', '151 F.Supp.3d 1294', '150 F.Supp.3d 228',
       '807 F.3d 952', '550 B.R. 676', '806 F.3d 1217', '541 B.R. 259',
       '146 F.Supp.3d 972', '146 F.Supp.3d 568', '541 B.R. 467'],
      dtype=object)

In [35]:
# Make a 'results' dataframe with one column: citation (string) 
results = pd.DataFrame({'citation': citations})

In [85]:
# Add a second column: response (string) (the result of the case.law API query)
results["response"] = results.apply(lambda x : requests.get(
    'https://api.case.law/v1/cases/?cite={}&full_case=true'.format(x['citation']),
    headers={'Authorization': " ".join(["Token",api_key])}
)._content.decode('UTF-8'), axis = 1)

In [86]:
# Add a third column: opinion (string) (the plain text of the opinion, as extracted from 'response')
results["opinion"] = results.apply(lambda x : json.loads(x["response"])['results'][0]['casebody']['data']['opinions'],
                                   axis = 1)

In [87]:
results

,citation,response,opinion
0,154 F.Supp.3d 189,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'Cathy Bissoon, United States Dist..."
1,154 F.Supp.3d 453,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'David Alan Ezra, Senior United St..."
2,162 F.Supp.3d 200,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'SHIRAA. SCHEINDLIN, UNITED STATES..."
3,151 F.Supp.3d 1043,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'Neil V. Wake, United States Distr..."
4,151 F.Supp.3d 1294,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'PAUL G..BYRON, UNITED STATES DIST..."
5,150 F.Supp.3d 228,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'SPATT, District Judge.', 'text': ..."
6,807 F.3d 952,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'LOKEN, Circuit Judge.', 'text': '..."
7,550 B.R. 676,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'Shira A. Scheindlin, U.S.D.J.:', ..."
8,806 F.3d 1217,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'MOLLOY, Senior District Judge:', ..."
9,541 B.R. 259,"{""count"":1,""next"":null,""previous"":null,""result...","[{'author': 'ERIC L. FRANK, CHIEF U.S. BANKRUP..."


In [89]:
results.to_csv("caselaw-fetch-results.csv")